In [ ]:
# Import packages
import os
import copy
import shutil
import numpy as np
import pandas as pd
from glob import glob
import dill
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
# from netneurotools import cluster
# from netneurotools import plotting as nnt_plotting
from nipype.interfaces.fsl.maths import BinaryMaths
from nipype.interfaces.fsl.utils import ImageMeants
import nibabel as nib
from stepmix.stepmix import StepMix
from nilearn.maskers import NiftiMasker, NiftiLabelsMasker
from nilearn import plotting, datasets, image
from sklearn.linear_model import RidgeClassifier, ElasticNet, LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.mixture import BayesianGaussianMixture
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import MNLogit
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from sklearn.cluster import SpectralClustering, KMeans
from scipy.spatial.distance import cdist
from statistics import mode
from scipy.stats import kruskal, pearsonr, spearmanr, zscore
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import statsmodels.formula.api as smf
from datetime import date
import re

today=str(date.today())

sns.set_palette('Paired')

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

In [ ]:
# Set paths and variables
home = '/gpfs/milgram/pi/gee_dylan/candlab/data'
hcpdata = home + '/mri/hcp_pipeline_preproc/shapes'
taskfiles = home + '/behavioral/shapes/task_design_trialwise'
datapath = '/gpfs/milgram/pi/gee_dylan/candlab/analyses/shapes/shapes_phenotyping'
fslpath = '/home/tjk33/project/SHAPES_task_act/out'
analysis = datapath + '/Analysis'
suffix = 'bold_8dv_resampled.nii.gz'
plt_out = analysis + '/Figures'
rois = '/gpfs/milgram/pi/gee_dylan/candlab/atlases/Mackey_vmPFC/VmPFC_symmetric_LMS/bin_niftis'

bv_df_orig = pd.read_csv(analysis + '/Behav_Dataset_n=131_2024-06-21.csv')
subjects = bv_df_orig['Subject']

In [ ]:
# Convert to categorical and standardize in behav df
bv_df_orig['sex'] = bv_df_orig['sex'].astype('category')
bv_df_orig['cdirisc_sum_z'] = zscore(bv_df_orig['cdirisc_sum'], nan_policy='omit')
bv_df_orig['years_education'] = bv_df_orig['years_education'].astype('category')
bv_df_orig['combined_income'] = bv_df_orig['combined_income'].astype('category')

### Define functions

In [ ]:
#Function using FSL tools via Nipype to average the two contrasts (runs 2 and 3) together
def avg_copes(sub, cope):
    if os.path.exists(datapath + '/{}/{}_{}_cope_averaged.nii.gz'.format(sub, sub, cope)):
            threat_cope = nib.load(datapath + '/{}/{}_{}_cope_averaged.nii.gz'.format(sub, sub, cope))
    else:
        BinaryMaths(in_file=fslpath + '/{}/testing_1/main.feat/stats/{}_cope.nii.gz'.format(sub, cope),
                          operand_file = fslpath + '/{}/testing_2/main.feat/stats/{}_cope.nii.gz'.format(sub, cope),
                          operation = 'add',
                          out_file = datapath + '/{}/{}_{}_cope_added.nii.gz'.format(sub, sub, cope)).run()
        
        BinaryMaths(in_file = datapath + '/{}/{}_{}_cope_added.nii.gz'.format(sub, sub, cope),
                   operand_value = 2,
                   operation = 'div',
                   out_file = datapath + '/{}/{}_{}_cope_averaged.nii.gz'.format(sub, sub, cope)).run()
    
    
    threat_cope_file = datapath + '/{}/{}_{}_cope_averaged.nii.gz'.format(sub, sub, cope)
    threat_cope = nib.load(threat_cope_file)
    
    return threat_cope_file, threat_cope

In [ ]:
# Function to get global mean of contrasts
def get_global_mean(sub, cope):
    # Load in copes
    cope_img1 = nib.load(fslpath + '/{}/testing_1/main.feat/stats/{}_cope.nii.gz'.format(sub, cope))
    cope_img2 = nib.load(fslpath + '/{}/testing_2/main.feat/stats/{}_cope.nii.gz'.format(sub, cope))
    copes_concat = np.concatenate([cope_img1.dataobj, cope_img2.dataobj], axis=2) # Concatenate along an axis
    
    #Compute means
    m1 = np.nanmean(copes_concat, axis=1)
    m2 = np.nanmean(m1, axis=1)
    mean_val = np.nanmean(m2) # Double checked this is correcct in FSL and with avg_copes function
    return mean_val

In [ ]:
def process_parcellate(sub, cope, cope_name, cope_mean, roi_file_list, names_list, demean, uniqueid):
    
    #Average runs 1 and 2 together
    cope_coefs_orig_file, cope_coefs_orig = avg_copes(sub, cope)

    if demean == True:
        print('Demeaning the data')
        # Demean the data
        cope_coefs_demeaned = cope_coefs_orig.get_fdata(caching='unchanged') 
        cope_coefs_demeaned[:, :, :] = cope_coefs_demeaned[:, :, :] - cope_mean #Will cache and update .dataobj in place
        cope_coefs = nib.Nifti1Image(cope_coefs_demeaned, cope_coefs_orig.affine)
        cope_coefs_demeaned_abs = np.absolute(cope_coefs_demeaned)
        cope_coefs_abs = nib.Nifti1Image(cope_coefs_demeaned_abs, cope_coefs_orig.affine)
        
        assert (cope_coefs_orig.get_fdata(caching='unchanged').all == cope_coefs_demeaned.all) == False
        assert (cope_coefs_demeaned.all == cope_coefs_demeaned_abs.all) == False
        
    else:
        cope_coefs = cope_coefs_orig
        cope_coefs_data_abs = np.absolute(cope_coefs_orig.get_fdata(caching='unchanged'))
        cope_coefs_abs = nib.Nifti1Image(cope_coefs_data_abs, cope_coefs_orig.affine)
    
    # Loop through ROIs list
    parc_cope = dict(left_hippocampus_Shen368 = [],
                      right_hippocampus_Shen368 = [],
                      left_amygdala_Shen368 = [],
                      right_amygdala_Shen368 = [],
                      Mackey_area14m_left = [],
                      Mackey_area14m_right = [],
                      Mackey_area24_left = [],
                      Mackey_area24_right = [],
                      Mackey_area25_left = [],
                      Mackey_area25_right = [],
                      Mackey_area32_left = [],
                      Mackey_area32_right = [])
    parc_cope_abs = deepcopy(parc_cope) 
    
    for j in range(0, len(roi_file_list)):
        
        roi=roi_file_list[j]
        name = names_list[j]
        print('Extracting data for {} {}...'.format(cope, name))

        ImageMeants(in_file=cope_coefs_orig_file,
                    mask = roi,
                    out_file = datapath + '/{}/{}_{}_{}_cope_meants.txt'.format(sub, sub, cope, name),
                    terminal_output = 'stream').run()

        cope_parc = pd.read_csv(datapath + '/{}/{}_{}_{}_cope_meants.txt'.format(sub, sub, cope, name),
                               sep = ' ', header=None).iloc[0,0]
        assert cope_parc != np.nan
        assert cope_parc != 0
        
        if len(roi_file_list) > 1:
            # Append output
            parc_cope[name].append(cope_parc)
            
        else:
            parc_cope = cope_parc
        
    return parc_cope 
        

In [ ]:
def parcellate_shen(sub, cope, cope_name, cope_mean, roi_file_list, names_list, demean, uniqueid):
    
    #Average runs 1 and 2 together
    cope_coefs_orig_file, cope_coefs_orig = avg_copes(sub, cope)

    if demean == True:
        print('Demeaning the data')
        # Demean the data
        cope_coefs_demeaned = cope_coefs_orig.get_fdata(caching='unchanged') 
        cope_coefs_demeaned[:, :, :] = cope_coefs_demeaned[:, :, :] - cope_mean #Will cache and update .dataobj in place
        cope_coefs = nib.Nifti1Image(cope_coefs_demeaned, cope_coefs_orig.affine)
        cope_coefs_demeaned_abs = np.absolute(cope_coefs_demeaned)
        cope_coefs_abs = nib.Nifti1Image(cope_coefs_demeaned_abs, cope_coefs_orig.affine)
        
        assert (cope_coefs_orig.get_fdata(caching='unchanged').all == cope_coefs_demeaned.all) == False
        assert (cope_coefs_demeaned.all == cope_coefs_demeaned_abs.all) == False
        
    else:
        cope_coefs = cope_coefs_orig
        cope_coefs_data_abs = np.absolute(cope_coefs_orig.get_fdata(caching='unchanged'))
        cope_coefs_abs = nib.Nifti1Image(cope_coefs_data_abs, cope_coefs_orig.affine)
    
    for loc, roi in enumerate(roi_file_list):
        name = names_list[loc]
        
        print('Parcellating {}'.format(name))
        raw_file1 = nib.load(datapath + '/{}/{}_task-shapes2_bold_8dv.nii.gz'.format(sub, sub))
        
        # Intialize masker
        masker = NiftiLabelsMasker(labels_img=roi, resampling_target=None, standardize=False, detrend=False)
        masker.fit(cope_coefs) #Fit on raw file
    
        print(cope_coefs.shape)
    
        # Apply our atlas to the Nifti object so we can pull out data from single parcels/ROIs
        cope_parc = masker.transform(cope_coefs) # Transform residuals
        
        #Transform absolute values
        cope_parc_abs = masker.transform(cope_coefs_abs) # Transform residuals abs value
        
        cope_parc_brain = masker.inverse_transform(cope_parc)
        nib.save(cope_parc_brain, datapath + '/{}/Shapes_{}Cope_{}_{}.nii.gz'.format(sub, cope_name, name, unique_id))
        
        # Plot  parcellation
        html_view = plotting.plot_roi(cope_parc_brain, # threshold=2, vmax=4,
                                      colorbar=True,
                                      title="Parcellated data for {}".format(sub))
        plt.show()
        if len(roi_file_list) > 1:
            # Append output
            parc_cope[name].append(cope_parc)
            
        else:
            parc_cope = cope_parc
            # parc_cope_abs = cope_parc_abs
        
    return parc_cope #, parc_cope_abs
        

### Compute whole-brain means

In [ ]:
# amin_means = []
# bmin_means = []
# amin_bmin_means = []

# for i in range(0, len(subjects)):
#     sub = subjects[i]
#     amin_mean = get_global_mean(sub, 'AMINUS')
#     bmin_mean = get_global_mean(sub, 'BMINUS')
#     amin_bmin = get_global_mean(sub, 'AMINUS-BMINUS')

#     amin_means.append([sub, amin_mean])
#     bmin_means.append([sub, bmin_mean])
#     amin_bmin_means.append([sub, amin_bmin])

In [ ]:
# threat_means = pd.DataFrame(amin_means, columns = ['Subject', 'aminus_mean_act'])
# safety_means = pd.DataFrame(bmin_means, columns = ['Subject', 'bminus_mean_act'])
# tvs_means = pd.DataFrame(amin_bmin_means, columns = ['Subject', 'aminus-bminus_mean_act'])

In [ ]:
# # Write means to CSV
# threat_means.to_csv(analysis + '/ThreatVsBaseline_WholeBrain_MeanActivation_n={}_{}.csv'.format(len(threat_means), today))
# safety_means.to_csv(analysis + '/SafetyVsBaseline_WholeBrain_MeanActivation_n={}_{}.csv'.format(len(safety_means), today))
# tvs_means.to_csv(analysis + '/ThreatVsSafety_WholeBrain_MeanActivation_n={}_{}.csv'.format(len(tvs_means), today))
# print(analysis + '/ThreatVsSafety_WholeBrain_MeanActivation_n={}_{}.csv'.format(len(tvs_means), today))

### Read in whole-brain means

In [ ]:
# *** Set unique identifier for this round of parcellations ***
unique_id = today + '_NotDemeaned'
# *************************************************************

In [ ]:
# # Read in means from date when computed
means_date = '2024-06-21'

threat_means = pd.read_csv(analysis + '/ThreatVsBaseline_WholeBrain_MeanActivation_n=131_{}.csv'.format(means_date), index_col=0)
safety_means = pd.read_csv(analysis + '/SafetyVsBaseline_WholeBrain_MeanActivation_n=131_{}.csv'.format(means_date), index_col=0)
tvs_means = pd.read_csv(analysis + '/ThreatVsSafety_WholeBrain_MeanActivation_n=131_{}.csv'.format(means_date), index_col=0)

### Parcellate Mackey ROIs

In [ ]:
# Define files list
vmpfc_files = glob(rois + '/left*resampled.nii.gz') + glob(rois + '/right*resampled.nii.gz') + glob(rois + '/*area14m_*resampled.nii.gz') + glob(rois + '/*area24_*resampled.nii.gz') + glob(rois + '/*area25_*resampled.nii.gz') + glob(rois + '/*area32_*resampled.nii.gz')

ho_dacc_files = glob(rois + '/HarvardOxford_dACC*.nii.gz')
aal_dacc_files = glob(rois + '/AAL3_dACC*.nii.gz')

# Define names list
vmpfc_names = pd.Series(vmpfc_files).str.replace(rois + '/','', regex=True).str.replace('_MNI_bin', '', regex=True).str.replace('.nii.gz', '', regex=True).str.replace('1_symmetric', 'Mackey', regex=True).str.replace('_resampled', '')

ho_dacc_names = pd.Series(ho_dacc_files).str.replace(rois + '/','', regex=True).str.replace('_maxprob_thr0_2mm_Binarized', '', regex=True).str.replace('.nii.gz', '', regex=True)

aal_dacc_names = pd.Series(aal_dacc_files).str.replace(rois + '/','', regex=True).str.replace('_156_bilat_bin', '', regex=True).str.replace('.nii.gz', '', regex=True)

In [ ]:
vmpfc_dict = dict(left_amygdala_Shen368 = [],
                  left_hippocampus_Shen368 = [],
                  right_amygdala_Shen368 = [],
                  right_hippocampus_Shen368 = [],
                  Mackey_area14m_left = [],
                  Mackey_area14m_right = [],
                  Mackey_area24_left = [],
                  Mackey_area24_right = [],
                  Mackey_area25_left = [],
                  Mackey_area25_right = [],
                  Mackey_area32_left = [],
                  Mackey_area32_right = [])

ho_dacc_dict = dict(HarvardOxford_dACC = [])
aal_dacc_dict = dict(AAL3_dACC=[])

In [ ]:
# Run function for all subjects

# ********* Define dict we want to run function for *********
roi_dict = vmpfc_dict
names_func_list = vmpfc_names
files_func_list = vmpfc_files
# ***********************************************************

subjects_added = []
parc_threat = deepcopy(roi_dict) 
parc_threat_abs = deepcopy(roi_dict) 
parc_safety = deepcopy(roi_dict)
parc_safety_abs = deepcopy(roi_dict)
parc_tvs = deepcopy(roi_dict)
parc_tvs_abs = deepcopy(roi_dict)
subs_added = []

for loc, sub in enumerate(subjects):
    print('*** Starting {}... ***'.format(sub))
    # Get means
    threat_mean = threat_means[threat_means['Subject'] == sub]['aminus_mean_act'].iloc[0]
    safety_mean = safety_means[safety_means['Subject'] == sub]['bminus_mean_act'].iloc[0]
    tvs_mean = tvs_means[tvs_means['Subject'] == sub]['aminus-bminus_mean_act'].iloc[0]

    # Run function
    threat_parc = process_parcellate(sub, 'AMINUS', 'ThreatVsBaseline', threat_mean, files_func_list, names_func_list, demean=False, uniqueid = unique_id)
    safety_parc = process_parcellate(sub, 'BMINUS', 'SafetyVsBaseline', safety_mean, files_func_list, names_func_list, demean=False, uniqueid = unique_id)
    tvs_parc = process_parcellate(sub, 'AMINUS-BMINUS', 'ThreatVsSafety', tvs_mean, files_func_list, names_func_list, demean=False, uniqueid = unique_id)
    
    for loc, name in enumerate(names_func_list):
        parc_threat[name].append(threat_parc[name][0]) #[name][0]
        # parc_threat_abs[name].append(threat_parc_abs[name])
        parc_safety[name].append(safety_parc[name][0]) #[name][0]
        # parc_safety_abs[name].append(safety_parc_abs[name])
        parc_tvs[name].append(tvs_parc[name][0]) #[name][0]
        # parc_tvs_abs[name].append(tvs_parc_abs[name])
        subs_added.append(sub)                                

In [ ]:
subswdata = pd.Series(pd.Series(subs_added).unique(), name='Subject', dtype='object')

In [ ]:
threat_data = pd.concat([subswdata,pd.DataFrame(parc_threat)], axis=1)
safety_data = pd.concat([subswdata,pd.DataFrame(parc_safety)], axis=1)
tvs_data= pd.concat([subswdata,pd.DataFrame(parc_tvs)], axis=1)

In [ ]:
# Check distributions of parcellated data
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 4))

sns.distplot(threat_data.iloc[:, 1], ax = ax1)
sns.distplot(safety_data.iloc[:, 1], ax = ax2)
sns.distplot(tvs_data.iloc[:, 1], ax = ax3)

In [ ]:
# Save to CSV
if roi_dict == vmpfc_dict:
    thr_file = analysis + '/ThreatVsBaseline_Subcortical_Mackey_vmPFC_CopeData_n={}_{}.csv'.format(len(threat_data), unique_id)
    threat_data.to_csv(thr_file)
    saf_file = analysis + '/SafetyVsBaseline_Subcortical_Mackey_vmPFC_CopeData_n={}_{}.csv'.format(len(safety_data), unique_id)
    safety_data.to_csv(saf_file)
    tvs_file = analysis + '/ThreatVsSafety_Subcortical_Mackey_vmPFC_CopeData_n={}_{}.csv'.format(len(threat_data), unique_id)
    tvs_data.to_csv(tvs_file)
elif roi_dict == ho_dacc_dict:
    thr_file = analysis + '/ThreatVsBaseline_Subcortical_HarvardOxford_dACC_CopeData_n={}_{}.csv'.format(len(threat_data), unique_id)
    threat_data.to_csv(thr_file)
    saf_file = analysis + '/SafetyVsBaseline_Subcortical_HarvardOxford_dACC_CopeData_n={}_{}.csv'.format(len(safety_data), unique_id)
    safety_data.to_csv(saf_file)
    tvs_file = analysis + '/ThreatVsSafety_Subcortical_HarvardOxford_dACC_CopeData_n={}_{}.csv'.format(len(threat_data), unique_id)
    tvs_data.to_csv(tvs_file)
elif roi_dict == aal_dacc_dict:
    thr_file = analysis + '/ThreatVsBaseline_Subcortical_AAL3_dACC_CopeData_n={}_{}.csv'.format(len(threat_data), unique_id)
    threat_data.to_csv(thr_file)
    saf_file = analysis + '/SafetyVsBaseline_Subcortical_AAL3_dACC_CopeData_n={}_{}.csv'.format(len(safety_data), unique_id)
    safety_data.to_csv(saf_file)
    tvs_file = analysis + '/ThreatVsSafety_Subcortical_AAL3_dACC_CopeData_n={}_{}.csv'.format(len(threat_data), unique_id)
    tvs_data.to_csv(tvs_file)

In [ ]:
print(thr_file)
print(saf_file)
print(tvs_file)

### Parcellate Shen Atlas

In [ ]:
# Load Shen 368 atlas
atlas = nib.load('/gpfs/milgram/pi/gee_dylan/lms233/Shen368/Shen_1mm_368_parcellation_resampled.nii.gz')
shen_filename = ['/gpfs/milgram/pi/gee_dylan/lms233/Shen368/Shen_1mm_368_parcellation_resampled.nii.gz']

shen_names = ['Shen368']

In [ ]:
coefs_date_shen = '2024-02-11'
subjects_loaded = []
parc_threat_shen = np.zeros((len(subjects), 368))
parc_safety_shen = np.zeros((len(subjects), 368))
parc_tvs_shen = np.zeros((len(subjects), 368))
parc_threat_shen_abs = np.zeros((len(subjects), 368))
parc_safety_shen_abs = np.zeros((len(subjects), 368))
parc_tvs_shen_abs = np.zeros((len(subjects), 368))

for i in range(0, len(subjects)):
    sub = subjects[i]
    print('Starting {}...'.format(sub))
    # Get means
    threat_mean = threat_means[threat_means['Subject'] == sub]['aminus_mean_act'].iloc[0]
    safety_mean = safety_means[safety_means['Subject'] == sub]['bminus_mean_act'].iloc[0]
    tvs_mean = tvs_means[tvs_means['Subject'] == sub]['aminus-bminus_mean_act'].iloc[0]

    # Run function
    threat_parc_shen = parcellate_shen(sub, 'AMINUS', 'ThreatVsBaseline', threat_mean, shen_filename, shen_names, demean=False, uniqueid = today + '_NotDemeaned')
    safety_parc_shen = parcellate_shen(sub, 'BMINUS', 'SafetyVsBaseline', safety_mean, shen_filename, shen_names, demean=False, uniqueid = today + '_NotDemeaned')
    tvs_parc_shen = parcellate_shen(sub, 'AMINUS-BMINUS', 'ThreatVsSafety', tvs_mean, shen_filename, shen_names, demean=False, uniqueid = today + '_NotDemeaned')

    # Save output
    parc_threat_shen[i,:] = threat_parc_shen 
    parc_safety_shen[i,:] = safety_parc_shen 
    parc_tvs_shen[i,:] = tvs_parc_shen 
    
    subjects_loaded.append(sub)

In [ ]:
subjects_loaded_s = subjects #pd.Series(subjects_loaded, name='Subject', dtype=str)

threat_shen = pd.concat([pd.Series(subjects_loaded_s, name='Subject', dtype = str),pd.DataFrame(parc_threat_shen, dtype=float)], axis=1).set_index('Subject')
safety_shen = pd.concat([pd.Series(subjects_loaded_s, name='Subject', dtype = str),pd.DataFrame(parc_safety_shen, dtype=float)], axis=1).set_index('Subject')
tvs_shen = pd.concat([pd.Series(subjects_loaded_s, name='Subject', dtype = str),pd.DataFrame(parc_tvs_shen, dtype=float)], axis=1).set_index('Subject')

In [ ]:
threat_shen.to_csv(analysis + '/ThreatVsBaseline_Shen368_Data_n={}_{}.csv'.format(len(threat_shen), unique_id))
safety_shen.to_csv(analysis + '/SafetyVsBaseline_Shen368_Data_n={}_{}.csv'.format(len(safety_shen), unique_id))
tvs_shen.to_csv(analysis + '/ThreatVsSafety_Shen368_Data_n={}_{}.csv'.format(len(threat_shen), unique_id))

In [ ]:
print(analysis + '/ThreatVsBaseline_Shen368_Data_n={}_{}.csv'.format(len(threat_shen), unique_id))
print(analysis + '/SafetyVsBaseline_Shen368_Data_n={}_{}.csv'.format(len(safety_shen), unique_id))
print(analysis + '/ThreatVsSafety_Shen368_Data_n={}_{}.csv'.format(len(threat_shen), unique_id))

### Read in already-parcellated data

In [ ]:
# # Read in de-meaned data
# parc_date = '2024-02-27'
# threat_vmpfc = pd.read_csv(analysis + '/ThreatVsBaseline_Subcortical_Mackey_Data_demeaned_n=124_{}.csv'.format(parc_date), index_col = 0).set_index('Subject')
# safety_vmpfc = pd.read_csv(analysis + '/SafetyVsBaseline_Subcortical_Mackey_Data_demeaned_n=124_{}.csv'.format(parc_date), index_col = 0).set_index('Subject')
# tvs_vmpfc = pd.read_csv(analysis + '/ThreatVsSafety_Subcortical_Mackey_Data_demeaned_n=124_{}.csv'.format(parc_date), index_col = 0).set_index('Subject')

# parc_date_shen = '2024-02-27'
# threat_shen = pd.read_csv(analysis + '/ThreatVsBaseline_Shen368_Data_demeaned_n=124_{}.csv'.format(parc_date_shen), index_col = 0)
# safety_shen = pd.read_csv(analysis + '/SafetyVsBaseline_Shen368_Data_demeaned_n=124_{}.csv'.format(parc_date_shen), index_col = 0)
# tvs_shen = pd.read_csv(analysis + '/ThreatVsSafety_Shen368_Data_demeaned_n=124_{}.csv'.format(parc_date_shen), index_col = 0)

In [ ]:
# Read in non-demeaned data
parc_date = '2024-06-21'
# ThreatVsBaseline_Subcortical_Mackey_Data_n=124_2024-03-21_NotDemeaned.csv'
threat_vmpfc = pd.read_csv(analysis + '/ThreatVsBaseline_Subcortical_Mackey_vmPFC_CopeData_n=132_{}_NotDemeaned.csv'.format(parc_date), index_col = 0).set_index('Subject').drop('sub-A647', axis=0).reset_index()
safety_vmpfc = pd.read_csv(analysis + '/SafetyVsBaseline_Subcortical_Mackey_vmPFC_CopeData_n=132_{}_NotDemeaned.csv'.format(parc_date), index_col = 0).set_index('Subject').drop('sub-A647', axis=0).reset_index()
tvs_vmpfc = pd.read_csv(analysis + '/ThreatVsSafety_Subcortical_Mackey_vmPFC_CopeData_n=132_{}_NotDemeaned.csv'.format(parc_date), index_col = 0).set_index('Subject').drop('sub-A647', axis=0).reset_index()

# # Shen data
# parc_date_shen = '2024-04-15'
# threat_shen = pd.read_csv(analysis + '/ThreatVsBaseline_Shen368_CopeData_n=124_NotDemeaned.csv'.format(parc_date_shen), index_col = 0)
# safety_shen = pd.read_csv(analysis + '/SafetyVsBaseline_Shen368_CopeData_n=124_NotDemeaned.csv'.format(parc_date_shen), index_col = 0)
# tvs_shen = pd.read_csv(analysis + '/ThreatVsSafety_Shen368_CopeData_n=124_NotDemeaned.csv'.format(parc_date_shen), index_col = 0)

# # Harvard/Oxford dACC data
# parc_date_dacc = '2024-06-20'
# threat_dacc = pd.read_csv(analysis + '/ThreatVsBaseline_Subcortical_AAL3_dACC_CopeData_n=124_{}_NotDemeaned.csv'.format(parc_date_dacc), index_col = 0)
# safety_dacc = pd.read_csv(analysis + '/SafetyVsBaseline_Subcortical_AAL3_dACC_CopeData_n=124_{}_NotDemeaned.csv'.format(parc_date_dacc), index_col = 0)
# tvs_dacc = pd.read_csv(analysis + '/ThreatVsSafety_Subcortical_AAL3_dACC_CopeData_n=124_{}_NotDemeaned.csv'.format(parc_date_dacc), index_col = 0)

### Regress covariates

In [ ]:
threat_data = threat_shen
safety_data = safety_shen
tvs_data = tvs_shen

In [ ]:
# Function to regress covariates

def regress_covariates(x_data_in, y_data_in, mean_activation, regress_mean):
    
    # Define matrix for regressed data
    regressed_data = np.ones(y_data_in.shape)

    if regress_mean == True:
        x_data_in = pd.concat([x_data_in, mean_activation], axis=1)
        print(len(x_data_in), len(mean_activation))
        assert len(x_data_in) == len(mean_activation)
    else:
        print("Not regressing mean activation")

    # Regress covariates column-wise
    for i in range(0, y_data_in.shape[1]):
        if i == .25*y_data_in.shape[1]:
            print("1/4 way through!")
        elif i == .5*y_data_in.shape[1]:
            print("1/2 way through!")
        elif i == .75*y_data_in.shape[1]:
            print("3/4 way through!")
        y_col = y_data_in.iloc[:,i]
        model = OLS(y_col, x_data_in)
        results = model.fit()
        print(results.summary())
        regressed_data[:, i] = results.resid
    
    # Put residuals in a data frame
    reg_df = pd.DataFrame(regressed_data, columns = y_data_in.columns)
    
    return reg_df

In [ ]:
# Do covariate regression
x_df = pd.merge(threat_data.reset_index()['Subject'], bv_df_orig[['Subject', 'eTIV', 'site_bin', 'mean_fd', 'age_at_scan']]) #Merge to ensure same subject order

x_data = deepcopy(x_df).set_index('Subject')
x_data['eTIV'] = zscore(x_data['eTIV'])
x_data['mean_fd'] = zscore(x_data['mean_fd'])
x_data['age_at_scan'] = zscore(x_data['age_at_scan'])
x_data = sm.add_constant(x_data) #Add intercept

In [ ]:
# Get variance inflation factor (code from https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python)
vifs = pd.DataFrame(np.linalg.inv(x_data.drop("const", axis=1).corr().to_numpy()).diagonal(), 
                 index=x_data.columns.drop("const"), 
                 columns=['VIF'])
# vifs
drop_vifs = np.where(vifs['VIF']>5)[0].tolist()

vifs

In [ ]:
# Random spot checks to ensure subjects in same order
# # ROI data
# assert x_df['Subject'].iloc[-1] == threat_data.index.iloc[-1]
# assert x_df['Subject'].iloc[-1] == tvs_data['Subject'].iloc[-1]
# assert x_df['Subject'].iloc[-1] == safety_data['Subject'].iloc[-1]

# assert x_df['Subject'].iloc[23] == threat_data['Subject'].iloc[23]
# assert x_df['Subject'].iloc[23] == tvs_data['Subject'].iloc[23]
# assert x_df['Subject'].iloc[23] == safety_data['Subject'].iloc[23]

# assert x_df['Subject'].iloc[78] == threat_data['Subject'].iloc[78]
# assert x_df['Subject'].iloc[78] == tvs_data['Subject'].iloc[78]
# assert x_df['Subject'].iloc[78] == safety_data['Subject'].iloc[78]

# Shen data
assert x_df['Subject'].iloc[-1] == threat_shen.index[-1]
assert x_df['Subject'].iloc[-1] == tvs_shen.index[-1]
assert x_df['Subject'].iloc[-1] == safety_shen.index[-1]

assert x_df['Subject'].iloc[23] == threat_shen.index[23]
assert x_df['Subject'].iloc[23] == tvs_shen.index[23]
assert x_df['Subject'].iloc[23] == safety_shen.index[23]

assert x_df['Subject'].iloc[78] == threat_shen.index[78]
assert x_df['Subject'].iloc[78] == tvs_shen.index[78]
assert x_df['Subject'].iloc[78] == safety_shen.index[78]

In [ ]:
# # Regress covariates from means (reset indices to avoid mis-matched index error (subs vs numbers))
# threat_shen_mean_reg = regress_covariates(x_data, threat_means.reset_index(drop=True), regress_mean=True)#.drop(['Subject'], axis=1)).set_index(threat_means['Subject'])
# safety_shen_mean_reg = regress_covariates(x_data, safety_means.reset_index(drop=True), regress_mean=True)#.drop(['Subject'], axis=1)).set_index(threat_means['Subject'])
# tvs_shen_mean_reg = regress_covariates(x_data, tvs_means.reset_index(drop=True), regress_mean=True)#.drop(['Subject'], axis=1)).set_index(threat_means['Subject'])

In [ ]:
# Do covariate regressions--Mackey + Subcort data
threat_reg_mackey = regress_covariates(x_data, threat_data.set_index('Subject'), threat_means.set_index('Subject'), regress_mean=True)
safety_reg_mackey = regress_covariates(x_data, safety_data.set_index('Subject'), safety_means.set_index('Subject'), regress_mean=True)
tvs_reg_mackey = regress_covariates(x_data, tvs_data.set_index('Subject'), tvs_means.set_index('Subject'), regress_mean=True)

In [ ]:
# Do covariate regressions--Shen data
threat_reg_shen = regress_covariates(x_data, threat_shen, threat_means.set_index('Subject'), regress_mean=True)
safety_reg_shen = regress_covariates(x_data, safety_shen, safety_means.set_index('Subject'), regress_mean=True)
tvs_reg_shen = regress_covariates(x_data, tvs_shen, tvs_means.set_index('Subject'), regress_mean=True)

In [ ]:
# Do covariate regressions--dACC data
threat_reg_dacc = regress_covariates(x_data, threat_data.set_index('Subject'), threat_means.set_index('Subject'), regress_mean=True)
safety_reg_dacc = regress_covariates(x_data, safety_data.set_index('Subject'), safety_means.set_index('Subject'), regress_mean=True)
tvs_reg_dacc = regress_covariates(x_data, tvs_data.set_index('Subject'), tvs_means.set_index('Subject'), regress_mean=True)

### Save Output

In [ ]:
# # write regressed means to csv (covariates regressed from single mean activation column)
# threat_shen_mean_reg.to_csv(analysis + '/ThreatVsBaseline_MeanActivation_regressed_n={}_{}.csv'.format(len(threat_shen_mean_reg), today))
# safety_shen_mean_reg.to_csv(analysis + '/SafetyVsBaseline_MeanActivation_regressed_n={}_{}.csv'.format(len(safety_shen_mean_reg), today))
# tvs_shen_mean_reg.to_csv(analysis + '/ThreatVsSafety_MeanActivation_regressed_n={}_{}.csv'.format(len(tvs_shen_mean_reg), today))

# print(analysis + '/ThreatVsBaseline_MeanActivation_regressed_n={}_{}.csv'.format(len(threat_vmpfc), today))
# print(analysis + '/SafetyVsBaseline_MeanActivation_regressed_n={}_{}.csv'.format(len(safety_vmpfc), today))
# print(analysis + '/ThreatVsSafety_MeanActivation_regressed_n={}_{}.csv'.format(len(threat_vmpfc), today))

In [ ]:
# Save output Shen data (covariates regressed from demeaned data)
today = str(date.today())

threat_reg_shen_df = pd.concat([x_df['Subject'], threat_reg_shen], axis=1).set_index('Subject')
threat_outfile = analysis + '/Regressed_ThreatVBaseline_Shen368_FSL_CopeBetas_n={}_{}.csv'.format(len(threat_reg_shen_df), unique_id)
threat_reg_shen_df.to_csv(threat_outfile)
print(threat_outfile)

safety_reg_shen_df = pd.concat([x_df['Subject'], safety_reg_shen], axis=1).set_index('Subject')
safety_outfile = analysis + '/Regressed_SafetyVBaseline_Shen368_FSL_CopeBetas_n={}_{}.csv'.format(len(safety_reg_shen_df), unique_id)
safety_reg_shen_df.to_csv(safety_outfile)
print(safety_outfile)

tvs_reg_shen_df = pd.concat([x_df['Subject'], tvs_reg_shen], axis=1).set_index('Subject')
tvs_outfile = analysis + '/Regressed_ThreatVSafety_Shen368_FSL_CopeBetas_n={}_{}.csv'.format(len(tvs_reg_shen_df), unique_id)
tvs_reg_shen_df.to_csv(tvs_outfile)
print(tvs_outfile)

In [ ]:
### MACKEY & SUBCORT DATA (covariates regressed from demeaned data)
# Save output
today = str(date.today())

threat_reg_mackey_df = pd.concat([x_df['Subject'], threat_reg_mackey], axis=1).set_index('Subject')
threat_outfile = analysis + '/Regressed_ThreatVBaseline_Mackey_FSL_CopeBetas_n={}_{}.csv'.format(len(threat_reg_mackey), unique_id)
threat_reg_mackey_df.to_csv(threat_outfile)
print(threat_outfile)

safety_reg_mackey_df = pd.concat([x_df['Subject'], safety_reg_mackey], axis=1).set_index('Subject')
safety_outfile = analysis + '/Regressed_SafetyVBaseline_Mackey_FSL_CopeBetas_n={}_{}.csv'.format(len(safety_reg_mackey), unique_id)
safety_reg_mackey_df.to_csv(safety_outfile)
print(safety_outfile)

tvs_reg_mackey_df = pd.concat([x_df['Subject'], tvs_reg_mackey], axis=1).set_index('Subject')
tvs_outfile = analysis + '/Regressed_ThreatVSafety_Mackey_FSL_CopeBetas_n={}_{}.csv'.format(len(tvs_reg_mackey), unique_id)
tvs_reg_mackey_df.to_csv(tvs_outfile)
print(tvs_outfile)

In [ ]:
### dACC DATA (covariates regressed from demeaned data)
# Save output
today = str(date.today())

if roi_dict == aal_dacc_dict:
    threat_reg_dacc_df = pd.concat([x_df['Subject'], threat_reg_dacc], axis=1).set_index('Subject')
    threat_outfile = analysis + '/Regressed_ThreatVBaseline_AAL3_dACC_FSL_CopeBetas_n={}_{}.csv'.format(len(threat_reg_dacc), unique_id)
    threat_reg_dacc_df.to_csv(threat_outfile)
    print(threat_outfile)
    
    safety_reg_dacc_df = pd.concat([x_df['Subject'], safety_reg_dacc], axis=1).set_index('Subject')
    safety_outfile = analysis + '/Regressed_SafetyVBaseline_AAL3_dACC_FSL_CopeBetas_n={}_{}.csv'.format(len(safety_reg_dacc), unique_id)
    safety_reg_dacc_df.to_csv(safety_outfile)
    print(safety_outfile)
    
    tvs_reg_dacc_df = pd.concat([x_df['Subject'], tvs_reg_dacc], axis=1).set_index('Subject')
    tvs_outfile = analysis + '/Regressed_ThreatVSafety_AAL3_dACC_FSL_CopeBetas_n={}_{}.csv'.format(len(tvs_reg_dacc), unique_id)
    tvs_reg_dacc_df.to_csv(tvs_outfile)
    print(tvs_outfile)
elif roi_dict == ho_dacc_dict:
    threat_reg_dacc_df = pd.concat([x_df['Subject'], threat_reg_dacc], axis=1).set_index('Subject')
    threat_outfile = analysis + '/Regressed_ThreatVBaseline_HarvardOxford_dACC_FSL_CopeBetas_n={}_{}.csv'.format(len(threat_reg_dacc), unique_id)
    threat_reg_dacc_df.to_csv(threat_outfile)
    print(threat_outfile)
    
    safety_reg_dacc_df = pd.concat([x_df['Subject'], safety_reg_dacc], axis=1).set_index('Subject')
    safety_outfile = analysis + '/Regressed_SafetyVBaseline_HarvardOxford_dACC_FSL_CopeBetas_n={}_{}.csv'.format(len(safety_reg_dacc), unique_id)
    safety_reg_dacc_df.to_csv(safety_outfile)
    print(safety_outfile)
    
    tvs_reg_dacc_df = pd.concat([x_df['Subject'], tvs_reg_dacc], axis=1).set_index('Subject')
    tvs_outfile = analysis + '/Regressed_ThreatVSafety_HarvardOxford_dACC_FSL_CopeBetas_n={}_{}.csv'.format(len(tvs_reg_dacc), unique_id)
    tvs_reg_dacc_df.to_csv(tvs_outfile)
    print(tvs_outfile)